In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install fastai==1.0.61

In [3]:
# !pip install timm

In [4]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from PIL import Image
import os
from glob import glob
import cv2
from tqdm import tqdm
import multiprocessing as mp
import pandas as pd
from matplotlib import pyplot as plt
from torchvision import transforms
import timm

import torch.nn.functional as F


In [5]:
# %cd /kaggle/working/
# !rm -rf Endoscopy-Image-Classification
# !git clone https://github.com/taindp98/Endoscopy-Image-Classification.git
# %cd './Endoscopy-Image-Classification/code'

In [6]:
from utils import interleave, de_interleave, show_batch, AverageMeter, show_grid, get_config
from learner import FixMatch, CoMatch, SupLearning
from dataset import get_data
from build import build_model

### Supervised Learning Re-produce

In [7]:
# config = get_config('./configs/kaggle_reproduce.yaml')
# df_anno = pd.read_csv(config['DATA']['ANNO'])
# df_unanno = None
# train_dl, valid_dl = get_data(config, df_anno, df_unanno, is_visual=True, is_reprod =True)
# model = build_model(config, is_pathology = True)

### Supervised Learning Abnormalities

In [8]:
# config = get_config('./configs/kaggle_supervised_abnorm.yaml')
# df_anno = pd.read_csv(config['DATA']['ANNO'])
# df_unanno = pd.read_csv(config['DATA']['UNANNO'])
# train_dl, valid_dl = get_data(config, 
#                               df_anno, 
#                               df_unanno, 
#                               is_full_sup = False, 
#                               is_visual=True, 
#                               type_semi = config.MODEL.TYPE_SEMI)

# ## generate prediction abnormalities
# unlabeled_dl = get_data(config, df_anno, df_unanno, is_visual=True, predict = True)
# model = build_model(config, is_pathology = False)

### Supervised Learning Pathologies

In [9]:
# config = get_config('./configs/kaggle_supervised_patho.yaml')
# df_anno = pd.read_csv(config['DATA']['ANNO'])
# df_unanno = None
# train_dl, valid_dl = get_data(config, df_anno, df_unanno, is_visual=True)
# model = build_model(config, is_pathology = True)

### Semi-supervised Learning

In [10]:
config = get_config('./configs/local_semisupervised_real.yaml')
df_anno = pd.read_csv(config['DATA']['ANNO'])
df_unanno = pd.read_csv(config['DATA']['UNANNO'])
train_dl, valid_dl = get_data(config, 
                              df_anno, 
                              df_unanno, 
                              is_full_sup = False, 
                              is_visual=True, 
                              type_semi = config.MODEL.TYPE_SEMI)

model = build_model(config, is_pathology = True)

Training mode: Full labeled and unlabeled data
Semi Transform mode: FixMatch


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\cls\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Admin\anaconda3\envs\cls\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Admin\anaconda3\envs\cls\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Admin\working\python\mine\endoscopic\code\dataset.py", line 204, in __getitem__
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
cv2.error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
if config.TRAIN.IS_SSL:
    if config.MODEL.TYPE_SEMI == 'FixMatch':
        classifier = FixMatch(model = model,
                          opt_func=config['TRAIN']['OPT_NAME'], 
                          device = device)
    else:
        classifier = CoMatch(model = model,
                          opt_func=config['TRAIN']['OPT_NAME'], 
                          device = device)
else:
    classifier = SupLearning(model = model,
                          opt_func=config['TRAIN']['OPT_NAME'], 
                          device = device)
classifier.get_dataloader(train_dl, valid_dl)
classifier.get_config(config)

In [ ]:
# classifier.save_checkpoint('/kaggle/working/')

In [ ]:
# classifier.load_checkpoint('/kaggle/input/checkpoint-abnormal/sup_densenet161_abnormal_224.pth')

In [ ]:
# classifier.evaluate_one(True, True, True)

In [ ]:
# out_pred = classifier.inference(unlabeled_dl)

In [ ]:
# out_pred

In [ ]:
# df_unanno['pred'] = list(out_pred)
# df_unanno['pred'].value_counts()

In [ ]:
# df_unanno.to_csv("/kaggle/working/full_unlabeled_predict.csv", header =True, index = False)

In [ ]:
classifier.fit()

In [ ]:
# classifier.save_checkpoint('/kaggle/working/')

In [ ]:
# classifier.load_checkpoint('/kaggle/working/06_29_2022_10_33_55_epoch_20.pth')

In [ ]:
# classifier.evaluate_one(True, True, True)